# Quickstart

First ensure you have [installed](http://elfi.readthedocs.io/en/stable/installation.html) Python 3.5 (or greater) and ELFI. After installation you can start using ELFI:

In [1]:
%load_ext autoreload
%autoreload 2
import elfi

ELFI includes an easy to use generative modeling syntax, where the generative model is specified as a directed acyclic graph (DAG). Let’s create two prior nodes:

In [2]:
mu = elfi.Prior('uniform', -2, 4)
sigma = elfi.Prior('uniform', 1, 4)

The above would create two prior nodes, a uniform distribution from -2 to 2 for the mean `mu` and another uniform distribution from 1 to 5 for the standard deviation `sigma`. All distributions from `scipy.stats` are available.

For likelihood-free models we typically need to define a simulator and summary statistics for the data. As an example, lets define the simulator as 30 draws from a Gaussian distribution with a given mean and standard deviation. Let's use mean and variance as our summaries:

In [3]:
import scipy.stats as ss
import numpy as np

def simulator(mu, sigma, batch_size=1, random_state=None):
    mu, sigma = np.atleast_1d(mu, sigma)
    res = ss.norm.rvs(mu[:, None], sigma[:, None], size=(batch_size, 5), random_state=random_state)
#     print(res)
    return res

def mean(y):
    return np.mean(y, axis=1)

def var(y):
    return np.var(y, axis=1)

Let’s now assume we have some observed data `y0` (here we just create some with the simulator):

In [4]:
# Set the generating parameters that we will try to infer
mean0 = 1
std0 = 3

# Generate some data (using a fixed seed here)
np.random.seed(20170525) 
y0 = simulator(mean0, std0)
# print(y0)

Now we have all the components needed. Let’s complete our model by adding the simulator, the observed data, summaries and a distance to our model:

In [5]:
# Add the simulator node and observed data to the model
sim = elfi.Simulator(simulator, mu, sigma, observed=y0)

# Add summary statistics to the model
S1 = elfi.Summary(mean, sim)
S2 = elfi.Summary(var, sim)

# Specify distance as euclidean between summary vectors (S1, S2) from simulated and
# observed data
d = elfi.Distance('euclidean', S1, S2)

If you have `graphviz` installed to your system, you can also visualize the model:

In [6]:
# Plot the complete model (requires graphviz)
# elfi.draw(d)

We can try to infer the true generating parameters `mean0` and `std0` above with any of ELFI’s inference methods. Let’s use ABC Rejection sampling and sample 1000 samples from the approximate posterior using threshold value 0.5:

In [7]:
import logging
logging.basicConfig(level=logging.INFO)

In [8]:
rej = elfi.Rejection(d, batch_size=1, seed=30052017)
res = rej.sample(3, threshold=10, n_sim=10)
# print(res)

INFO:elfi.methods.parameter_inference:Submitting batch 0
INFO:elfi.methods.parameter_inference:{'d': array([ 3.86207825]), 'mu': array([-1.72990121]), 'sigma': array([ 1.18611568])}
INFO:elfi.methods.parameter_inference:Submitting batch 1
INFO:elfi.methods.parameter_inference:{'d': array([ 7.98150327]), 'mu': array([-0.75308713]), 'sigma': array([ 3.85439944])}
INFO:elfi.methods.parameter_inference:Submitting batch 2
INFO:elfi.methods.parameter_inference:{'d': array([ 27.97657017]), 'mu': array([-0.75530055]), 'sigma': array([ 4.67184471])}
INFO:elfi.methods.parameter_inference:Submitting batch 3
INFO:elfi.methods.parameter_inference:{'d': array([ 13.32199351]), 'mu': array([-1.09842533]), 'sigma': array([ 3.43857164])}
INFO:elfi.methods.parameter_inference:Submitting batch 4
INFO:elfi.methods.parameter_inference:{'d': array([ 1.72094821]), 'mu': array([ 1.2628132]), 'sigma': array([ 3.50565518])}


----------
+++++
10
=====
+++++
10
=====
None
+++++
4
=====
+++++
4
=====
None
+++++
4
=====
+++++
4
=====
None
+++++
5
=====
+++++
5
=====
None
+++++
7
=====
+++++
7
=====
None
+++++
5
=====


Let's plot also the marginal distributions for the parameters:

In [9]:
import matplotlib.pyplot as plt
res.plot_marginals()
# plt.show()

array([<matplotlib.axes._subplots.AxesSubplot object at 0x10883f630>,
       <matplotlib.axes._subplots.AxesSubplot object at 0x1088e4a20>], dtype=object)